# SPRINT-3

# Splitting the data into Training Data & Test Data

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

In [52]:
print(X.shape, X_train.shape, X_test.shape)

(400, 24) (320, 24) (80, 24)


# Model Training

# Logistic Regression

In [53]:
model1 = LogisticRegression()

In [54]:
#training the LogisticRegression model with Training Data

In [55]:
model1.fit(X_train, Y_train)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Model Evaluation

# Accuracy Score

In [56]:
#accuracy on training data
X_train_prediction = model1.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [57]:
print('Accuracy on Training data ', training_data_accuracy)

Accuracy on Training data  0.99375


In [58]:
#accuracy on test data
X_test_prediction = model1.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [59]:
print('Accuracy on Training data ', test_data_accuracy)

Accuracy on Training data  0.975


# Building a Predictive System

In [60]:
input_data = (9,53.0,90.0,1.02,2.0,0.0,1,1,1,0,70.0,107.0,7.2,114.0,3.7,9.5,29,12100,3.7,1,1,0,0,1)

In [61]:
# change the input data to a numpy array
input_data_as_numpy_array= np.asarray(input_data)

# reshape the numpy array as we are predicting for only on instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model1.predict(input_data_reshaped)
print(prediction)

if (prediction[0]== 0):
  print('The Person does not have Chronic Kidney Disease')
else:
  print('The Person has Chronic Kidney Disease')

[1]
The Person has Chronic Kidney Disease


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


# Save the Model

In [63]:
filename = 'kidney_disease_model.sav'
pickle.dump(model1, open(filename,'wb'))

In [64]:
#loading the saved model
loaded_model = pickle.load(open('kidney_disease_model.sav','rb'))

In [65]:
for column in X.columns:
    print(column)

Age
Blood_Pressure
Specific_Gravity
Albumin
Sugar
Red_Blood_Cells
Pus_Cells
Puss_Cell_Clumps
Bacteria
Blood_Gulcose_Random
Blood_Urea
Serum_Creatinine
Sodium
Potassium
Haemoglobin
Packed_Cell_Volume
White_Blood_Cell_Count
Red_Blood_Cell_Count
Hypertension
Diabetes_Mellitus
Coronary_Artery_Disease
Appetite
Peda_Edema
Aanemia


In [66]:
!pip install ibm_watson_machine_learning

In [67]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"CJcHQrCI1TnphrVTyUTmEf_HWjJPtT48RRAo5eTdM7cs"
                  }
client = APIClient(wml_credentials)

In [68]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [69]:
space_uid = guid_from_space_name(client, 'models')
print("Space UID = " + space_uid)

Space UID = b35966c8-39e9-4912-ab26-5a08194f0a48


In [70]:
client.set.default_space(space_uid)

'SUCCESS'

In [71]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [89]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [90]:
model_details = client.repository.store_model(model = model1,meta_props={
    client.repository.ModelMetaNames.NAME:"ckd",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


In [91]:
model_id

'02955551-bd38-4b56-94bc-46e315588ae5'

In [95]:
input_data = (9,53.0,90.0,1.02,2.0,0.0,1,1,1,0,70.0,107.0,7.2,114.0,3.7,9.5,29,12100,3.7,1,1,0,0,1)

In [96]:
prediction = model1.predict(input_data_reshaped)
print(prediction)

[1]
